In [1]:
#install touch
!pip install torch
!pip install torchvision

In [2]:
# importing the libraries
import pandas as pd
import numpy as np

# for reading and displaying images
from skimage.io import imread
import matplotlib.pyplot as plt
%matplotlib inline

from numpy.lib.stride_tricks import as_strided

# for creating validation set
from sklearn.model_selection import train_test_split

# for evaluating the model
from sklearn.metrics import accuracy_score
from tqdm import tqdm

# PyTorch libraries and modules
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
from  torch.nn.modules.pooling import _MaxPoolNd
from torch.autograd import Variable
from torch.nn import Linear, ReLU, CrossEntropyLoss, Sequential, Conv2d, MaxPool2d, Module, Softmax, BatchNorm2d, Dropout
from torch.optim import Adam, SGD


In [3]:
# All those for custom pooling 
from torch import Tensor
from typing import List, Optional
from torch.nn.modules.utils import _single, _pair, _triple 
from torch.nn import functional as F
from torch.nn.common_types import _size_any_t, _size_1_t, _size_2_t, _size_3_t, _ratio_3_t, _ratio_2_t

In [4]:
#Define Hyperparameters
input_size = 784 # img_size = (28,28) ---> 28*28=784 in total
hidden_size = 500 # number of nodes at hidden layer
num_classes = 10 # number of output classes discrete range [0,9]
num_epochs = 5 # number of times which the entire dataset is passed throughout the model
batch_size = 100 # the size of input data took for one iteration
lr = 1e-3 # size of step 

In [5]:
#Downloading MNIST data

train_data = dsets.MNIST(root = './data', train = True,
                        transform = transforms.ToTensor(), download = True)

test_data = dsets.MNIST(root = './data', train = False,
                       transform = transforms.ToTensor())


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw




Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw




Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw




Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [6]:
#Loading the data

train_gen = torch.utils.data.DataLoader(dataset = train_data,
                                             batch_size = batch_size,
                                             shuffle = True)

test_gen = torch.utils.data.DataLoader(dataset = test_data,
                                      batch_size = batch_size, 
                                      shuffle = False)

Fuzzy Pooling Imeplementation From Scratch


In [7]:
## Fuzzy membership function u1
def member_fun_u1(pixel):
  r_max = 6
  d = 3
  c = 1
  res_pixel = 0

  if pixel>d:
    res_pixel = 0
  elif c<=pixel and pixel<=d:
    p_1 = d - pixel
    p_2 = d-c
    res_pixel = p_1/p_2
  elif pixel < c:
    res_pixel = 1

  return res_pixel

## Fuzzy membership function u2
def member_fun_u2(pixel):
  r_max = 6
  a = 1.5
  m = 3
  b =  m + a
  res_pixel = 0

  if pixel<=a:
    res_pixel = 0
  elif a<=pixel and pixel<=m:
    p_1 = pixel - a
    p_2 = m-a
    res_pixel = p_1/p_2
  elif m<pixel and pixel<b:
    p_1 = b - pixel
    p_2 = b - m
    res_pixel = p_1/p_2
  elif pixel >= b:
    res_pixel = 0
  
  return res_pixel

## Fuzzy membership function u3
def member_fun_u3(pixel):
  r_max = 6
  r = 3
  q = 4.5
  res_pixel = 0

  if pixel < r:
    res_pixel = 0
  elif r <= pixel and pixel <= q:
    p_1 = pixel - r
    p_2 = q - r
    res_pixel = p_1/p_2
  elif pixel > q:
    res_pixel = 1

  return res_pixel

In [8]:
def get_max_membership_sum_function(a, b, c):
  if a >= b and a>=c:
    return 1
  elif b>=a and b>=c:
    return 2
  else :
    return 3  

In [9]:
def fuzzy_pooling_2d(AA, kernel_size, stride):
    
    # convert tensor to numpy 
    A = torch.Tensor.cpu(AA).detach().numpy()[:,:,:,:]

    # define the image size , channel size
    x1,x2,x3,x4 = A.shape

    new_mat_size1 = (x3 - kernel_size)//stride + 1
    new_mat_size2 = (x4 - kernel_size)//stride + 1

    #print(new_mat_size1)

    pool_ar = np.zeros((x1,x2,new_mat_size1,new_mat_size2))

    for image_i in range(0, x1):
        for channel_j in range(0,x2):
            mat = A[image_i][channel_j]
            
            # now do the pooling work

            # divide the matrix into k*k kernel
            output_shape = ((mat.shape[0] - kernel_size)//stride + 1,(mat.shape[1] - kernel_size)//stride + 1)            
            kernel_size1 = (kernel_size, kernel_size)    
                      
            mat_k = as_strided(mat, shape = output_shape + kernel_size1, strides = (stride*mat.strides[0],stride*mat.strides[1]) + mat.strides)
            mat_k = mat_k.reshape(-1, * kernel_size1)
            
            number_of_kernel = mat_k.shape[0]
            
            fuzzy_mat = np.zeros(number_of_kernel)
            
            for k in range(0, number_of_kernel):
              ur1 = np.zeros((kernel_size,kernel_size))
              ur2 = np.zeros((kernel_size,kernel_size))
              ur3 = np.zeros((kernel_size,kernel_size))

              u1_sum , u2_sum , u3_sum = 0, 0, 0

              for i in range(0, kernel_size) :
                for j in range(0, kernel_size) :
                  # euation 8
                  ur1[i][j] =  member_fun_u1(mat_k[k][i][j])
                  ur2[i][j] =  member_fun_u2(mat_k[k][i][j])
                  ur3[i][j] =  member_fun_u3(mat_k[k][i][j])

                  # equation 9
                  u1_sum = u1_sum + ur1[i][j]
                  u2_sum = u2_sum + ur2[i][j]
                  u3_sum = u3_sum + ur3[i][j]

                  #Fuzzy Algebric Sum
                  #Equation 9 Modification 29-06-2021
                  #u1_sum = (u1_sum + ur1[i][j])-(u1_sum*ur1[i][j])
                  #u2_sum = (u2_sum + ur2[i][j])-(u2_sum*ur2[i][j])
                  #u3_sum = (u3_sum + ur3[i][j])-(u3_sum*ur3[i][j])
              
              #check max membership sum value equation 10
              id = get_max_membership_sum_function(u1_sum, u2_sum, u3_sum)
              
              # equation 11
              kernel_value = 0

              if id == 1:
                if u1_sum != 0:
                  for i in range(0, kernel_size) :
                    for j in range(0, kernel_size) :
                      tmp = ur1[i][j] * mat_k[k][i][j]
                      kernel_value = kernel_value + tmp
                  kernel_value = kernel_value / u1_sum
              elif id == 2:
                if u2_sum != 0:
                  for i in range(0, kernel_size) :
                    for j in range(0, kernel_size) :
                      tmp = ur2[i][j] * mat_k[k][i][j]
                      kernel_value = kernel_value + tmp
                  kernel_value = kernel_value / u2_sum
              else :
                if u3_sum != 0:
                  for i in range(0, kernel_size) :
                    for j in range(0, kernel_size) :
                      tmp = ur3[i][j] * mat_k[k][i][j]
                      kernel_value = kernel_value + tmp
                  kernel_value = kernel_value / u3_sum
              
              fuzzy_mat[k] = kernel_value

            fuzzy_mat = fuzzy_mat.reshape(output_shape)
            pool_ar[image_i][channel_j] = fuzzy_mat;
    
    pool_ar = torch.from_numpy(pool_ar);    
    return pool_ar

In [10]:
class FuzzyPool2d_from_scratch(_MaxPoolNd):
    kernel_size: _size_2_t
    stride: _size_2_t

    def forward(self, input: Tensor) -> Tensor:
        return fuzzy_pooling_2d(input, self.kernel_size, self.stride)


In [11]:
class Net(Module):   
    def __init__(self):
        super(Net, self).__init__()

        self.cnn_layers = Sequential(
            # Defining a 2D convolution layer
            Conv2d(1, 4, kernel_size=3, stride=1, padding=1),
            BatchNorm2d(4),
            ReLU(inplace=True),
            FuzzyPool2d_from_scratch(kernel_size=3, stride=2),
            
            # Defining another 2D convolution layer
            Conv2d(4, 4, kernel_size=3, stride=1, padding=1),
            BatchNorm2d(4),
            ReLU(inplace=True),
            FuzzyPool2d_from_scratch(kernel_size=3, stride=2),
        )

        self.linear_layers = Sequential(
            Linear(4 * 6 * 6, 10)
        )

    # Defining the forward pass    
    def forward(self, x):
        x = self.cnn_layers(x)
        x = x.view(x.size(0), -1)
        x = self.linear_layers(x)
        return x

In [12]:
# defining the model
model = Net()

# defining the optimizer
optimizer = Adam(model.parameters(), lr=0.07)

# defining the loss function
loss_function = CrossEntropyLoss()

model = model.double()

# checking if GPU is available
if torch.cuda.is_available():
    model = model.cuda()
    loss_function = loss_function.cuda()

In [13]:
# Training the model
for epoch in range(num_epochs):
  for i ,(images,labels) in enumerate(train_gen):
    #convert torch formate
    images = images.reshape(100, 1, 28, 28)
    #images  = torch.from_numpy(images)

    images = Variable(images)
    labels = Variable(labels)
    
    optimizer.zero_grad()
    outputs = model(images.double())
    loss = loss_function(outputs, labels)
    loss.backward()
    optimizer.step()
    
    if (i+1) % 100 == 0:
      print('Epoch [%d/%d], Step [%d/%d], Loss: %.4f'
                 %(epoch+1, num_epochs, i+1, len(train_data)//batch_size, loss.item()))

Epoch [1/5], Step [100/600], Loss: 0.3139
Epoch [1/5], Step [200/600], Loss: 0.3266
Epoch [1/5], Step [300/600], Loss: 0.2345
Epoch [1/5], Step [400/600], Loss: 0.2206
Epoch [1/5], Step [500/600], Loss: 0.3044
Epoch [1/5], Step [600/600], Loss: 0.1436
Epoch [2/5], Step [100/600], Loss: 0.2249
Epoch [2/5], Step [200/600], Loss: 0.2318
Epoch [2/5], Step [300/600], Loss: 0.3874
Epoch [2/5], Step [400/600], Loss: 0.3766
Epoch [2/5], Step [500/600], Loss: 0.3468
Epoch [2/5], Step [600/600], Loss: 0.3058
Epoch [3/5], Step [100/600], Loss: 0.1926
Epoch [3/5], Step [200/600], Loss: 0.2925
Epoch [3/5], Step [300/600], Loss: 0.3063
Epoch [3/5], Step [400/600], Loss: 0.3081
Epoch [3/5], Step [500/600], Loss: 0.2606
Epoch [3/5], Step [600/600], Loss: 0.2523
Epoch [4/5], Step [100/600], Loss: 0.2044
Epoch [4/5], Step [200/600], Loss: 0.4242
Epoch [4/5], Step [300/600], Loss: 0.2419
Epoch [4/5], Step [400/600], Loss: 0.2357
Epoch [4/5], Step [500/600], Loss: 0.2934
Epoch [4/5], Step [600/600], Loss:

In [15]:
#Evaluating the accuracy of the model
correct = 0
total = 0
for images,labels in test_gen:
  #convert torch formate
  images = images.reshape(100, 1, 28, 28)
  images = Variable(images)
  labels = labels
  
  output = model(images.double())
  _, predicted = torch.max(output,1)
  correct += (predicted == labels).sum()
  total += labels.size(0)

print('Accuracy of the model: %.3f %%' %((100*correct)/(total+1)))

Accuracy of the model: 92.321 %
